In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import time
import os

## Reservoir
Here I give a walkthrough of how to use PyTorch for constructing a reservoir, essentially a randomized recurrent neural network, for classification. A more in-depth implementation which could be good for reference is [EchoTorch](https://github.com/nschaetti/EchoTorch)

The main steps I use are to initialize a RNN with standard nodes, replace the weight matrix only the readout layer is trained.

There are many other parameters to tune.

In [23]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as utils


# set random seeds for reproducibility
torch.manual_seed(0)
np.random.seed(0)

First in order to use PyTorch effectively we want to it up to use the availabe GPU device

In [24]:
# Setup the ability to run on a GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

cuda:0


## Constructing the reservoir
Here we create a recurrent neural network, set its internal weights according to the Echo State Property and then freeze all the weights except the output.
Additionally aother simple way of doing this is by removing the final layer and running the data through the reservoir and then training a classifier on these node output values.

In [25]:
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F
import torch.optim as optim

class ESN(nn.Module):
    def __init__(self, input_size, n_hidden):
        super(ESN, self).__init__()
        self.n_hidden = n_hidden

        self.hidden = nn.RNN(input_size, n_hidden, batch_first=True, bias=False)
        self.fc = nn.Linear(n_hidden, 1)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(1, x.size(0), self.n_hidden).to(device)

        # Forward propagate
        out, h_n = self.hidden(x, h0)  # out: tensor of shape (batch_size, seq_length, n_hidden)
        out = self.fc(out[:,-1,:])
        return out

def create_reservoir_matrix(size=(10,10), spectral_radius=0.9, sparsity=0.5):
    """
    inputs:
    size: square matrix representing the size of teh reservoir connections
    spectral_radius: largest eigenvalue in reservoir matrix should be <1
    sparsity: connectivity of matrix, 1.0 indicates full connection
    """
    # generate uniformly random from U[0,1] *2 -1 makes it U[-1,1]
    W_res = torch.rand(size)*2-1

    # create a sparse mask array then multiply reservoir weights by sparse mask
    # sparse mask is done by generating uniform[0,1] then setting True <=sparsity
    W_res = W_res*(torch.rand_like(W_res) <= sparsity).type(W_res.dtype)

    # scale the matrix to have the desired spectral radius
    W_res = W_res*spectral_radius/(np.max(np.abs(np.linalg.eigvals(W_res))))

    return W_res


def epoch_accuracy(loader, total_size, split='train'):
    """
    evaluates the accuracy of the model on the dataset
    """
    model.eval()
    # Measure the accuracy for the entire dataset
    correct = 0
    total = 0
    for inputs, labels in loader:
        inputs = inputs.float().to(device)
        labels = labels.float().to(device)
        outputs = model(inputs)
        predicted = inputs.prod(0)
        total += labels.size(0)
        correct += (predicted == outputs).sum()

    print(f'Accuracy of the model on the {total_size} {split} sequences: {float(correct) / total:3.1%}')

    return float(correct) / total

class MultDataset(Dataset):
	def __init__(self, size):
		self.size = size
		self.data = np.random.random((size, 2))*2-1
		self.data = np.hstack((self.data, np.multiply(self.data[:, 0], self.data[:, 1]).reshape((size, 1))))

	def __len__(self):
		return self.size

	def __getitem__(self, item):
		if torch.is_tensor(item):
			item = item.tolist()

		return torch.from_numpy(self.data[item, :2]), torch.from_numpy(np.asarray(self.data[item, 2]))

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [27]:
# set random seeds for reproducibility
torch.manual_seed(0)
np.random.seed(0)

# Hyper Parameters
num_epochs = 10
batch_size_train = 64
batch_size_test = 1024
learning_rate = 1e-3

In [28]:
train_dataset = MultDataset(1024)
test_dataset = MultDataset(1024)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size_train, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size_test, shuffle=False)

In [29]:
# input parameters
input_size = 2

# Hyper-parameters
n_hidden = 50
batch_size = batch_size_train
learning_rate = 0.001

# create the internal weight matrix
W_res = create_reservoir_matrix(size=(n_hidden, n_hidden), spectral_radius=0.9, sparsity=1.0)

model = ESN(input_size, n_hidden)

# set the internal hidden weight matrix as the reservoir values
model.hidden.weight_hh_l0 = nn.Parameter(W_res, requires_grad=False)
input_scale = 0.5
model.hidden.weight_ih_l0 = nn.Parameter((torch.rand((n_hidden,input_size))*2-1)*input_scale, requires_grad=False)

# move to the GPU
model.to(device)

# Freeze all hidden layers so no gradient update occurs
for param in model.hidden.parameters():
    param.requires_grad = False

# only need the gradient for the fully connected layer, weight_decay adds l2 norm
optimizer = torch.optim.Adam(model.fc.parameters(), lr=learning_rate, weight_decay=0)

# CrossEntropyLoss takes care of the Softmax evaluation
criterion = nn.MSELoss()

In [33]:
train_acc = []
test_acc = []

num_epochs = 30
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        # reshape to (batch_size, time_step, input size)
        inputs = inputs.view(1,batch_size_train,input_size)
        inputs = inputs.float().to(device)
        labels = labels.float().to(device)

        # Forward pass
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Optimize
        optimizer.step()

plt.plot(train_acc)
plt.plot(test_acc)
plt.legend(['train_acc','test_acc'])#%%

TabError: inconsistent use of tabs and spaces in indentation (<ipython-input-33-6a7d0d85374a>, line 29)